In [ ]:
import gzip
import shutil
import os
import pandas as pd
import json

In [15]:
for i in os.listdir():
  if 'json' in i:
    with gzip.open(i, 'rb') as f_in:
          with open(i.replace('.gz',''), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [11]:
receipts = pd.read_json('receipts.json',lines=True)
brands = pd.read_json('brands.json',lines=True)
users = pd.read_json('users.json',lines=True)
'''
receipts = read_json('receipts.json')
users = read_json('users.json')
brands = read_json('brands.json')
'''

Tried opening the original '.json.gz' files via python code, ran into a 'Expected object or value' error. Also tried adding comma to end of line but still got an error. Since this did not work so I used online sites to open the file - to get them to be viewable. For specificity: getting an error on line two for all files. For brands.json: " Unexpected non-whitespace character after JSON at position 229 (line 2 column 1) ". I have added the json files I used in the data files folder. Decided to convert to csv via website due to issues. I understand that this would not be possible in an actually data pipeline but to complete this assignment, this is my current solution.

In [27]:
# Read the CSV file
receipts_df = pd.read_csv('receipts.csv')

brands_df = pd.read_csv('brands.csv')

users_df = pd.read_csv('users.csv')

# Print column names
print(" Receipts Column names:")
print(receipts_df.columns)
print(" Brands Column names:")
print(brands_df.columns)
print(" Users Column names:")
print(users_df.columns)

 Receipts Column names:
Index(['_id__$oid', 'bonusPointsEarned', 'bonusPointsEarnedReason',
       'createDate__$date', 'dateScanned__$date', 'finishedDate__$date',
       'modifyDate__$date', 'pointsAwardedDate__$date', 'pointsEarned',
       'purchaseDate__$date', 'purchasedItemCount',
       'rewardsReceiptItemList__deleted', 'rewardsReceiptItemList__brandCode',
       'rewardsReceiptItemList__competitorRewardsGroup',
       'rewardsReceiptItemList__barcode',
       'rewardsReceiptItemList__competitiveProduct',
       'rewardsReceiptItemList__description',
       'rewardsReceiptItemList__discountedItemPrice',
       'rewardsReceiptItemList__finalPrice',
       'rewardsReceiptItemList__itemNumber',
       'rewardsReceiptItemList__itemPrice',
       'rewardsReceiptItemList__metabriteCampaignId',
       'rewardsReceiptItemList__originalReceiptItemText',
       'rewardsReceiptItemList__needsFetchReview',
       'rewardsReceiptItemList__originalFinalPrice',
       'rewardsReceiptItemList

In [50]:
# Convert CSV file to dataframe
receipts_df = pd.read_csv('receipts.csv')

brands_df = pd.read_csv('brands.csv')

users_df = pd.read_csv('users.csv')

In [52]:
### Perform checks on receipts ###

numeric_columns = ['bonusPointsEarned', 'pointsEarned', 'purchasedItemCount', 'totalSpent']
for column in numeric_columns:
    receipts_df[column] = pd.to_numeric(receipts_df[column], errors='coerce')  # Coerce = set as NaN

# Check id field is unique & not null
print("Total receipt rows:", len(receipts_df['_id__$oid']))
print("Unique receipt id values:", receipts_df['_id__$oid'].nunique())
print("Null receipt id values:", receipts_df['_id__$oid'].isna().sum())

Total receipt rows: 2114
Unique receipt id values: 322
Null receipt id values: 1792


Possible null values due to how file was created.

In [53]:
# Check if userID field is also not null or if there is diff b/w id and user id in data
print("Total rows with null userID:", receipts_df['userId'].isna().sum())

Total rows with null userID: 1792


Depending on how the nested json was flattened/normalized (since I did not do it using code im not sure how it was done) this can cause certain rows to be null when they should not be. Can also cause the table to be inflated -- would need to add code to specify which node/field to flatten. --Noticed same amount as null id values.

In [55]:
print(receipts_df[numeric_columns].describe().loc[['count', 'mean', 'min', 'max']])
# Look at mean, min, and max values

       bonusPointsEarned  pointsEarned  purchasedItemCount   totalSpent
count         242.000000    267.000000          278.000000   298.000000
mean          260.710744    667.020599           10.035971    46.711544
min             5.000000      0.000000            0.000000     0.000000
max           750.000000   9449.800000          335.000000  1177.840000


In [58]:
# Check order of dates is correct -- if there issue, would have to look into it further
# finished date after any other date, scan prior to points awarded)
date_fields = ['createDate__$date', 'dateScanned__$date', 'modifyDate__$date', 'pointsAwardedDate__$date',
               'purchaseDate__$date', 'finishedDate__$date']

for date_field in date_fields[1:]:
    print(f"Number of records where {date_field} is before createDate: ",
          (receipts_df[date_field] < receipts_df['createDate__$date']).sum())
print('')

for date_field in date_fields[:-1]:
    print(f"Number of records where {date_field} is after to finishedDate: ",
          (receipts_df[date_field] > receipts_df['finishedDate__$date']).sum())

print('')
print("Number of records where dateScanned is before purchaseDate: ",
      (receipts_df['dateScanned__$date'] < receipts_df['purchaseDate__$date']).sum()) #might be issue with date/upload/ or data

Number of records where dateScanned__$date is before createDate:  0
Number of records where modifyDate__$date is before createDate:  0
Number of records where pointsAwardedDate__$date is before createDate:  0
Number of records where purchaseDate__$date is before createDate:  264
Number of records where finishedDate__$date is before createDate:  0

Number of records where createDate__$date is after to finishedDate:  0
Number of records where dateScanned__$date is after to finishedDate:  0
Number of records where modifyDate__$date is after to finishedDate:  75
Number of records where pointsAwardedDate__$date is after to finishedDate:  0
Number of records where purchaseDate__$date is after to finishedDate:  3

Number of records where dateScanned is before purchaseDate:  9


In [59]:
# Brands

# Checking id field is unique & not null --same checks as before
print("Total brand rows:", len(brands_df['_id__$oid']))
print("Unique brand _id values:", brands_df['_id__$oid'].nunique())
print("Null brand _id values:", brands_df['_id__$oid'].isna().sum())

# Checking one-to-one relationship
print(brands_df[['category', 'categoryCode']].groupby('category').nunique())

# Checking if barcodes and brand codes are unique
print(brands_df[['barcode', '_id__$oid']].groupby('barcode').size().sort_values())
print(brands_df[['brandCode', '_id__$oid']].groupby('brandCode').size().sort_values())

Total brand rows: 1167
Unique brand _id values: 1167
Null brand _id values: 0
                             categoryCode
category                                 
Baby                                    1
Baking                                  1
Beauty                                  0
Beauty & Personal Care                  0
Beer Wine Spirits                       1
Beverages                               1
Bread & Bakery                          1
Breakfast & Cereal                      0
Candy & Sweets                          1
Canned Goods & Soups                    0
Cleaning & Home Improvement             1
Condiments & Sauces                     0
Dairy                                   0
Dairy & Refrigerated                    1
Deli                                    0
Frozen                                  1
Grocery                                 1
Health & Wellness                       1
Household                               0
Magazines                               

In [61]:
# Users

# Checking id field is unique & not null --same checks as before
print("Total user rows:", len(users_df['_id__$oid']))
print("Unique user _id values:", users_df['_id__$oid'].nunique())
print("Null user _id values:", users_df['_id__$oid'].isna().sum())
print("Number of duplicated user records: ", users_df.duplicated().sum())

Total user rows: 495
Unique user _id values: 212
Null user _id values: 0
Number of duplicated user records:  283


In [62]:
# For SQL Query:
print(receipts_df.groupby('rewardsReceiptStatus').agg({'totalSpent': 'mean', 'purchasedItemCount': 'sum'}))

                      totalSpent  purchasedItemCount
rewardsReceiptStatus                                
FINISHED               48.077269              2664.0
FLAGGED                64.821429                95.0
PENDING                28.570000                 0.0
REJECTED                7.604167                31.0
SUBMITTED                    NaN                 0.0


Assuming Finished is accepted: Accepted is greater